# Simple Imitation Learning in MineRL
This tutorial contains a simple example of how to build a imitation-learning based agent that can solve the MineRLNavigateDense-v0 environment. For more information about that environment, see this [MineRL Docs](http://minerl.io/docs/environments/index.html#minerlnavigatedense-v0).

For more Imitation Learning algorithms, like a Dagger in Tensorflow, see that Github repo, [Dagger](https://github.com/zsdonghao/Imitation-Learning-Dagger-Torcs).

Parts of this tutorial are based on code by [Arthur Juliani](https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724).

In [1]:
from __future__ import division

import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
%matplotlib inline
import matplotlib.pyplot as plt
import math

try:
    xrange = xrange
except:
    xrange = range
    
env_name = 'MineRLTreechop-v0'
data_path = '/media/kimbring2/6224AA7924AA5039/minerl_data'

/home/kimbring2/minerl_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kimbring2/minerl_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kimbring2/minerl_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kimbring2/minerl_env/lib/python3.6/site-packages/tensorflow/pyth

### Setting up our Neural Network agent
This time we will be using a Policy neural network that takes observations, passes them through a single hidden layer, and then produces a probability of choosing a left/right movement. To learn more about this network, see [Andrej Karpathy's blog on Policy Gradient networks](http://karpathy.github.io/2016/05/31/rl/).

In [2]:
H = 1024

tf.reset_default_graph()

if (env_name == 'MineRLTreechop-v0'):
    state = tf.placeholder(shape=[None,64,64,3], dtype=tf.float32)
elif (env_name == 'MineRLNavigateDense-v0'):
    state = tf.placeholder(shape=[None,64,64,4], dtype=tf.float32)

conv1 = slim.conv2d(inputs=state, num_outputs=32, kernel_size=[8,8], stride=[4,4], padding='VALID', 
                    biases_initializer=None, activation_fn=tf.nn.relu)
conv2 = slim.conv2d(inputs=conv1, num_outputs=64, kernel_size=[4,4], stride=[2,2], padding='VALID', 
                    biases_initializer=None, activation_fn=tf.nn.relu)
conv3 = slim.conv2d(inputs=conv2, num_outputs=64, kernel_size=[3,3], stride=[1,1], padding='VALID', 
                    biases_initializer=None, activation_fn=tf.nn.relu)

convFlat = slim.flatten(conv3)
#print("convFlat: " + str(convFlat))

if (env_name == 'MineRLTreechop-v0'):
    W = tf.get_variable("W", shape=[H,6],
               initializer=tf.contrib.layers.xavier_initializer())
    score = tf.matmul(convFlat, W)
    probability = tf.nn.softmax(score)
    real_action = tf.placeholder(shape=[None,6], dtype=tf.int32)
elif (env_name == 'MineRLNavigateDense-v0'):
    W = tf.get_variable("W", shape=[H,4],
               initializer=tf.contrib.layers.xavier_initializer())
    score = tf.matmul(convFlat, W)
    probability = tf.nn.softmax(score)
    real_action = tf.placeholder(shape=[None,4], dtype=tf.int32)
    
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=real_action, 
                                                              logits=score))
tf.summary.scalar('loss', loss)
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)

merged = tf.summary.merge_all()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



# Train
MineRL package provides a human playing dataset for improving effiency of traning. At first, we are going to train our network by this dataset and use pretrained network for Reinforcement Learning. I assure it will reduce traing time tremendously. 

For more information about that dataset, see this [MineRL Dataset Docs](http://minerl.io/docs/tutorials/data_sampling.html).

### Running the Agent and Environment

In [3]:
import minerl
data = minerl.data.make(env_name, data_path)

print("test")

init = tf.global_variables_initializer()
restore = False
with tf.Session() as sess:
    rendering = False
    sess.run(init)
    saver = tf.train.Saver(max_to_keep=5)
    train_writer = tf.summary.FileWriter('/home/kimbring2/MineRL/train_summary/' + env_name, sess.graph)
    
    if restore == True:
        path = '/home/kimbring2/MineRL/model/' + env_name
        ckpt = tf.train.get_checkpoint_state(path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    
    episode_count = 0
    for current_state, action, reward, next_state, done in data.sarsd_iter(num_epochs=500, max_sequence_len=10):
        #print("action: " + str(action))
        
        #print("current_state['equipped_items']: " + str(current_state['equipped_items']))
        length = (current_state['pov'].shape)[0]

        action_list = []
        states_list = []
        for i in range(0, length):
            if (env_name == 'MineRLTreechop-v0'):
                state_concat = current_state['pov'][i].astype(np.float32) / 255.0 - 0.5
            elif (env_name == 'MineRLNavigateDense-v0'):
                pov = current_state['pov'][i].astype(np.float32) / 255.0 - 0.5
                compass = current_state['compassAngle'][i]
                #print("compass: " + str(compass))
                
                #print("np.ones(shape=list(pov.shape[:-1]): " + str(np.ones(shape=list(pov.shape[:-1]))))
                compass_channel = np.ones(shape=list(pov.shape[:-1]) + [1], dtype=np.float32) * compass
                #print("compass_channel: " + str(compass_channel))
                
                compass_channel /= 180.0
                
                state_concat = np.concatenate([pov, compass_channel], axis=-1)

                
            if (env_name == 'MineRLNavigateDense-v0'):
                if (action['camera'][i][1] < 0):
                    action_ = [1, 0, 0, 0]
                elif (action['camera'][i][1] > 0):
                    action_ = [0, 1, 0, 0]
                else:
                    if (action['jump'][i] == 0):
                        action_ = [0, 0, 1, 0]
                    else:
                        action_ = [0, 0, 0, 1]
            elif (env_name == 'MineRLTreechop-v0'):
                if (action['camera'][i][1] < 0):
                    action_ = [1, 0, 0, 0, 0, 0]
                elif (action['camera'][i][1] > 0):
                    action_ = [0, 1, 0, 0, 0, 0]
                elif (action['camera'][i][0] < 0):
                    action_ = [0, 0, 1, 0, 0, 0]
                elif (action['camera'][i][0] > 0):
                    action_ = [0, 0, 0, 1, 0, 0]
                else:
                    if ( (action['jump'][i] == 0) & (action['forward'][i] == 0) ):
                        action_ = [0, 0, 0, 0, 1, 0]
                    elif ( (action['jump'][i] == 1) & (action['forward'][i] == 1) ):
                        action_ = [0, 0, 0, 0, 0, 1]                    
                                                       
            states_list.append(state_concat)
            action_list.append(action_)
        
        episode_count = episode_count + 1
        
        #state_train = (np.zeros([1,H]), np.zeros([1,H]))
        feed_dict = {state:np.stack(states_list, 0),
                     real_action:np.stack(action_list, 0)
                    }
        
        #if episode_count % 100 == 0:
        summary, _ = sess.run([merged, train_step], feed_dict=feed_dict)
        train_writer.add_summary(summary, episode_count)

        sess.run(train_step, feed_dict=feed_dict)
        
        if episode_count % 10 == 0:
            model_path = '/home/kimbring2/MineRL/model/' + env_name
            saver.save(sess, model_path + '/model-' + str(episode_count) + '.cptk')
            print("Saved Model")

test
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model

Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Save

Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Save

Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Save

Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Save

...

Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Save

Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Saved Model
Save

Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File 

Saved Model
Saved Model


KeyboardInterrupt: 

In [ ]:
'''
import minerl
import gym
env = gym.make('MineRLNavigateDense-v0')

obs  = env.reset()
done = False
net_reward = 0

while not done:
    action = env.action_space.noop()

    action['camera'] = [0, -10]
    action['back'] = 0
    action['forward'] = 1
    action['jump'] = 1
    action['attack'] = 1

    obs, reward, done, info = env.step(
        action)

    net_reward += reward
    print("Total reward: ", net_reward)
'''

In [ ]:
#import minerl
#data = minerl.data.make('MineRLNavigateDense-v0', '/home/kimbring2/MineRL/data/')

# Test

In [3]:
import minerl
import gym

env = gym.make('MineRLTreechop-v0')
obs = env.reset()

In [5]:
import random

init = tf.global_variables_initializer()
with tf.Session() as sess:
# Launch the graph
    rendering = False
    sess.run(init)
    saver = tf.train.Saver(max_to_keep=5)
    train_writer = tf.summary.FileWriter('/home/kimbring2/MineRL/train_summary', sess.graph)
    
    #print('Loading Model...')
    #path = '/home/kimbring2/MineRL/model/' + env_name
    #ckpt = tf.train.get_checkpoint_state(path)
    #saver.restore(sess, ckpt.model_checkpoint_path)
    
    env.init()
    obs = env.reset()
    net_reward = 0
    for i in range(0, 500000):
        if (env_name == 'MineRLTreechop-v0'):
            state_concat = obs['pov'].astype(np.float32) / 255.0 - 0.5
        elif (env_name == 'MineRLNavigateDense-v0'):
            pov = obs['pov'].astype(np.float32) / 255.0 - 0.5
            compass = obs['compassAngle']
            compass_channel = np.ones(shape=list(pov.shape[:-1]) + [1], dtype=np.float32) * compass
            compass_channel /= 180.0
            state_concat = np.concatenate([pov, compass_channel], axis=-1)
        
        e = 0.01
        
        action_probability = sess.run(probability, feed_dict={state:[state_concat]})
        print("action_probability: " + str(action_probability))
        if np.random.rand(1) >= e:
            action_index = np.argmax(action_probability)
        else:
            if (env_name == 'MineRLNavigateDense-v0'):
                action_index = random.randint(0,4)
            elif (env_name == 'MineRLTreechop-v0'):
                action_index = random.randint(0,6)
            
        action = env.action_space.noop()
        if (env_name == 'MineRLNavigateDense-v0'):
            if (action_index == 0):
                action['camera'] = [0, -10]
                action['jump'] = 0
                action['forward'] = 1
                action['sprint'] = 1
            elif (action_index == 1):
                action['camera'] = [0, 10]
                action['jump'] = 0
                action['forward'] = 1
                action['sprint'] = 1
            elif (action_index == 2):
                action['camera'] = [0, 0]
                action['jump'] = 0
                action['forward'] = 1
                action['sprint'] = 1
            else:
                action['camera'] = [0, 0]
                action['jump'] = 1
                action['forward'] = 1
                action['sprint'] = 1
        elif (env_name == 'MineRLTreechop-v0'):
            if (action_index == 0):
                action['camera'] = [0, -10]
                action['jump'] = 0
                action['forward'] = 1
                action['attack'] = 1
                action['sprint'] = 0
            elif (action_index == 1):
                action['camera'] = [0, 10]
                action['jump'] = 0
                action['forward'] = 1
                action['attack'] = 1
                action['sprint'] = 0
            elif (action_index == 2):
                action['camera'] = [-10, 0]
                action['jump'] = 0
                action['forward'] = 1
                action['attack'] = 1
                action['sprint'] = 0
            elif (action_index == 3):
                action['camera'] = [10, 0]
                action['jump'] = 0
                action['forward'] = 1
                action['attack'] = 1
                action['sprint'] = 0
            elif (action_index == 4):
                action['camera'] = [0, 0]
                action['jump'] = 0
                action['forward'] = 0
                action['attack'] = 1
                action['sprint'] = 0
            else:
                action['camera'] = [0, 0]
                action['jump'] = 1
                action['forward'] = 1
                action['attack'] = 1
                action['sprint'] = 0

                
            action['back'] = 0
            action['left'] = 0
            action['right'] = 0
            action['sprint'] = 0

        obs1, reward, done, info = env.step(action)
        
        if done == True:
            break
                
        obs = obs1

        net_reward += reward
    print("Total reward: ", net_reward)

action_probability: [[0.16819865 0.15656818 0.16091618 0.16928844 0.17296357 0.17206502]]
action_probability: [[0.16872552 0.15331563 0.16154423 0.1708176  0.17408825 0.17150868]]
action_probability: [[0.16734146 0.15411998 0.16247398 0.17027286 0.17530881 0.17048292]]
action_probability: [[0.16789693 0.15651992 0.15965329 0.16888022 0.17476052 0.1722891 ]]
action_probability: [[0.16505669 0.15349355 0.1583317  0.17201406 0.17734727 0.17375667]]
action_probability: [[0.16229992 0.15669706 0.15982316 0.16886812 0.17742929 0.17488244]]
action_probability: [[0.1611329  0.16105378 0.15914594 0.16679788 0.17807831 0.1737912 ]]
action_probability: [[0.16068459 0.16122665 0.15962635 0.1682439  0.17779048 0.17242803]]
action_probability: [[0.1632486  0.16335458 0.15693411 0.16762851 0.17220707 0.17662714]]
action_probability: [[0.163821   0.15612723 0.15851974 0.16649486 0.17501163 0.1800255 ]]
action_probability: [[0.16535215 0.15903665 0.15815262 0.16864704 0.16985603 0.1789555 ]]
action_pro

action_probability: [[0.16420613 0.15335457 0.16324818 0.17406099 0.17975003 0.16538005]]
action_probability: [[0.16434678 0.1530629  0.1633109  0.17403336 0.17975405 0.16549201]]
action_probability: [[0.1643278  0.15293074 0.1636372  0.17456193 0.17927355 0.16526875]]
action_probability: [[0.16434892 0.15308835 0.16353793 0.17430405 0.1794558  0.16526498]]
action_probability: [[0.16478096 0.15272814 0.16329159 0.17393872 0.17969069 0.16556984]]
action_probability: [[0.16483858 0.1527243  0.16339494 0.1740981  0.17942813 0.16551597]]
action_probability: [[0.16480356 0.15300511 0.16327825 0.17408071 0.17920269 0.16562966]]
action_probability: [[0.16480638 0.1530976  0.16349214 0.17366652 0.17930776 0.16562957]]
action_probability: [[0.16456789 0.15300345 0.1630592  0.17433964 0.1794784  0.16555144]]
action_probability: [[0.16464876 0.14745213 0.15782638 0.17518091 0.18242583 0.17246598]]
action_probability: [[0.16771026 0.1485743  0.15575823 0.1732672  0.18231776 0.17237231]]
action_pro

action_probability: [[0.1652532  0.1478702  0.16159774 0.17224824 0.18346739 0.16956323]]
action_probability: [[0.16536766 0.14860028 0.16151595 0.17164695 0.18333219 0.16953693]]
action_probability: [[0.16499239 0.14866267 0.16175935 0.17159021 0.18270206 0.17029332]]
action_probability: [[0.16499102 0.14873928 0.16167468 0.17162272 0.18261585 0.17035647]]
action_probability: [[0.16510804 0.14864753 0.16177137 0.17167908 0.18272889 0.17006512]]
action_probability: [[0.16506746 0.14851691 0.16181965 0.17194642 0.18250315 0.17014636]]
action_probability: [[0.16493958 0.1486333  0.16164231 0.17187421 0.18243338 0.17047714]]
action_probability: [[0.16520983 0.1482753  0.16203266 0.17214312 0.1828084  0.16953067]]
action_probability: [[0.16518596 0.14847158 0.16164139 0.17174949 0.18309925 0.16985235]]
action_probability: [[0.16518198 0.14849165 0.16175722 0.17172597 0.18305553 0.16978765]]
action_probability: [[0.16532667 0.14857039 0.16166557 0.17176148 0.18308719 0.16958871]]
action_pro

action_probability: [[0.16263284 0.15408568 0.15846616 0.17648475 0.17911245 0.1692182 ]]
action_probability: [[0.1638969  0.15402825 0.15980619 0.17588623 0.17755584 0.16882661]]
action_probability: [[0.1637851  0.15419884 0.15982188 0.1761286  0.17757653 0.16848901]]
action_probability: [[0.16400892 0.15387249 0.16007608 0.1760774  0.17789467 0.16807047]]
action_probability: [[0.16383201 0.15392464 0.16008745 0.17617683 0.17800304 0.16797602]]
action_probability: [[0.16348656 0.15424705 0.1599754  0.17627609 0.1780741  0.16794084]]
action_probability: [[0.16371664 0.15404373 0.16021912 0.17635033 0.17799538 0.16767474]]
action_probability: [[0.16373074 0.15398468 0.16024078 0.17627351 0.1779856  0.16778466]]
action_probability: [[0.16378576 0.15401615 0.16022171 0.17616658 0.17782544 0.1679844 ]]
action_probability: [[0.16376407 0.15404715 0.16005567 0.17623296 0.1778536  0.16804655]]
action_probability: [[0.16381884 0.15388502 0.1598087  0.17630944 0.17809989 0.16807812]]
action_pro

action_probability: [[0.16531718 0.15478888 0.1554303  0.17410268 0.18046953 0.16989143]]
action_probability: [[0.16525634 0.15479465 0.15543443 0.17418152 0.1808176  0.16951545]]
action_probability: [[0.16539131 0.15434587 0.15577212 0.17423888 0.18040624 0.16984557]]
action_probability: [[0.16538207 0.15470777 0.15576781 0.17391798 0.18022306 0.1700013 ]]
action_probability: [[0.16590714 0.15465066 0.15598513 0.1732145  0.17998305 0.17025946]]
action_probability: [[0.16598967 0.15430824 0.15602036 0.17366934 0.17976996 0.17024244]]
action_probability: [[0.16574377 0.15436609 0.15592153 0.17384958 0.17980959 0.17030947]]
action_probability: [[0.16589795 0.15471701 0.15581985 0.17375578 0.17988862 0.16992077]]
action_probability: [[0.1659695  0.15470387 0.15580966 0.17371504 0.18006447 0.16973746]]
action_probability: [[0.16552922 0.15482196 0.15577915 0.17376871 0.17994435 0.17015652]]
action_probability: [[0.16548383 0.15484181 0.15597777 0.17375103 0.17956713 0.17037845]]
action_pro

...

action_probability: [[0.16454391 0.15689844 0.1563257  0.17013122 0.18301477 0.16908588]]
action_probability: [[0.16460851 0.15681724 0.15627363 0.17023198 0.18294516 0.16912352]]
action_probability: [[0.16477415 0.15669163 0.15617956 0.17015377 0.18306968 0.16913122]]
action_probability: [[0.1647355  0.15686686 0.15599664 0.16976981 0.1832286  0.1694026 ]]
action_probability: [[0.16445522 0.15712439 0.15628994 0.16985835 0.18306313 0.16920893]]
action_probability: [[0.16437827 0.1571107  0.15628639 0.1699583  0.18311267 0.16915366]]
action_probability: [[0.16441031 0.15709788 0.15643866 0.16983265 0.18298118 0.16923934]]
action_probability: [[0.16452004 0.15710661 0.1564808  0.1699164  0.18295592 0.1690202 ]]
action_probability: [[0.16471846 0.15669635 0.15636112 0.16987157 0.18304472 0.16930778]]
action_probability: [[0.16460268 0.15680335 0.1563752  0.16980222 0.18308583 0.16933072]]
action_probability: [[0.16452122 0.15691102 0.15642676 0.16969159 0.18312326 0.1693262 ]]
action_pro

action_probability: [[0.1643203  0.15777938 0.15636696 0.1691243  0.18243627 0.16997278]]
action_probability: [[0.16467993 0.1571914  0.15628472 0.16955763 0.18267135 0.16961497]]
action_probability: [[0.16453259 0.15701665 0.15635662 0.16994806 0.18294424 0.16920178]]
action_probability: [[0.16444597 0.15671586 0.1565066  0.17009465 0.18324074 0.16899617]]
action_probability: [[0.16472504 0.15644665 0.15674146 0.16984569 0.18321902 0.16902219]]
action_probability: [[0.16469537 0.15673676 0.15682988 0.16960803 0.18294124 0.16918874]]
action_probability: [[0.16472498 0.1570238  0.15678659 0.16933367 0.18283609 0.16929486]]
action_probability: [[0.16484423 0.15682557 0.15677842 0.16938041 0.18315865 0.16901271]]
action_probability: [[0.16472632 0.15714127 0.15651177 0.16924116 0.18320677 0.1691727 ]]
action_probability: [[0.16468033 0.15705638 0.15668331 0.16931082 0.18299973 0.16926946]]
action_probability: [[0.16478376 0.15672763 0.15674989 0.1693786  0.18276066 0.16959949]]
action_pro

action_probability: [[0.16424973 0.15739788 0.15688665 0.16893014 0.18255405 0.16998155]]
action_probability: [[0.16437215 0.1571164  0.15682778 0.1690788  0.18295088 0.169654  ]]
action_probability: [[0.16437215 0.1571164  0.15682778 0.1690788  0.18295088 0.169654  ]]
action_probability: [[0.16420828 0.15749122 0.15683544 0.1690485  0.18247508 0.1699415 ]]
action_probability: [[0.16414443 0.15751615 0.15681237 0.16915828 0.1822624  0.17010635]]
action_probability: [[0.1642003  0.15734598 0.15704595 0.16923574 0.18237053 0.16980158]]
action_probability: [[0.16429637 0.15735105 0.15692377 0.16913347 0.18246096 0.1698344 ]]
action_probability: [[0.16385292 0.1576051  0.15669903 0.16923782 0.18256737 0.17003772]]
action_probability: [[0.16402446 0.1577293  0.15663481 0.16908127 0.18246043 0.17006981]]
action_probability: [[0.16390146 0.15789449 0.15677038 0.16900337 0.18236485 0.17006549]]
action_probability: [[0.16424395 0.15750307 0.1566964  0.16914812 0.18265599 0.16975243]]
action_pro

action_probability: [[0.16246854 0.15809886 0.1565075  0.16890213 0.18361367 0.17040935]]
action_probability: [[0.1624311  0.15783778 0.15680787 0.16925353 0.183513   0.1701567 ]]
action_probability: [[0.16186881 0.15861952 0.15654042 0.16939873 0.18274985 0.17082265]]
action_probability: [[0.16215251 0.15867285 0.15761884 0.16824463 0.18263565 0.17067549]]
action_probability: [[0.16263774 0.15794723 0.15630122 0.1684926  0.18407331 0.17054786]]
action_probability: [[0.16241288 0.15842636 0.15588437 0.16812317 0.18432105 0.17083213]]
action_probability: [[0.16219306 0.1583679  0.15490034 0.16794075 0.18515079 0.1714472 ]]
action_probability: [[0.1631097  0.15754299 0.15539862 0.16881843 0.18424882 0.1708814 ]]
action_probability: [[0.16259773 0.15762855 0.15523775 0.16857383 0.18486273 0.17109944]]
action_probability: [[0.16320962 0.15752943 0.15543444 0.16811717 0.18460897 0.17110044]]
action_probability: [[0.16318105 0.15744714 0.15500857 0.16843222 0.18514766 0.1707834 ]]
action_pro

KeyboardInterrupt: 

As you can see, the network not only does much better than random actions, but achieves the goal of 200 points per episode, thus solving the task!